<a href="https://colab.research.google.com/github/ML-HW-SYS/a3-anya-23-ct/blob/main/2_conv1d_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1D Convolution on GPU

## 1. Set-up

In [ ]:
# Mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Make sure your token is stored in a txt file at the location below.
# This way there is no risk that you will push it to your repo
# Never share your token with anyone, it is basically your github password!
with open('/content/gdrive/MyDrive/ece5545/token.txt') as f:
    token = f.readline().strip()
# Use another file to store your github username
with open('/content/gdrive/MyDrive/ece5545/git_username.txt') as f:
    handle = f.readline().strip()

In [ ]:
# Clone your github repo
YOUR_TOKEN = token
YOUR_HANDLE = handle
BRANCH = "main"

# %mkdir /content/gdrive/MyDrive/ece5545
%cd /content/gdrive/MyDrive/ece5545
# !git clone https://{YOUR_TOKEN}@github.com/ML-HW-SYS/a3-{YOUR_HANDLE}.git
%cd /content/gdrive/MyDrive/ece5545/a3-{YOUR_HANDLE}
!git checkout {BRANCH}
!git pull
%cd /content/gdrive/MyDrive/ece5545

PROJECT_ROOT = f"/content/gdrive/MyDrive/ece5545/a3-{YOUR_HANDLE}"

/content/gdrive/MyDrive/ece5545
/content/gdrive/MyDrive/ece5545/a3-anya-23-ct
Already on 'main'
Your branch is up to date with 'origin/main'.
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), 4.64 KiB | 94.00 KiB/s, done.
From https://github.com/ML-HW-SYS/a3-anya-23-ct
   1141f92..3f2272a  main       -> origin/main
Updating 1141f92..3f2272a
Fast-forward
 1_conv1d_cpu.ipynb | 568 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++-
 1 file changed, 567 insertions(+), 1 deletion(-)
/content/gdrive/MyDrive/ece5545


In [ ]:
# This extension reloads all imports before running each cell
%load_ext autoreload
%autoreload 2

In [ ]:
!ls {PROJECT_ROOT}

1_conv1d_cpu.ipynb  3-conv1d_fpga.ipynb  5_conv2d_dw_gpu.ipynb	README.md  tests
2_conv1d_gpu.ipynb  4_gemm_gpu.ipynb	 leaderboard_id.txt	src


## 2. Install TVM

In [ ]:
!pip install tlcpack-nightly-cu102 -f https://tlcpack.ai/wheels

Looking in links: https://tlcpack.ai/wheels
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 428.6/428.6 MB 2.0 MB/s eta 0:00:00


## 3. Implement `make_conv1d_gpu_scheduler_func` function in `src.ops`

In that function, you are required to implemented 1D convolution and use TVM to optimize it.
Let $x \in \mathbb{R}^m$ and $y \in \mathbb{R}^n$, then
$$
\operatorname{conv1d}(x, y)_i = \sum_{j=-\infty}^{\infty} x[j]y[i-j], \forall i \in \{0, 1, \dots, m + n - 1\}
$$

Please use zero padding and unit stride. Please see the numpy convolution function for more detail: [link](https://numpy.org/doc/stable/reference/generated/numpy.convolve.html).

The `make_conv1d_gpu_scheduler_func` takes $m$ and $n$, which are the size of the two 1D input array.
You should return both the TVM scheduler and the TVM opterator for
1. Input $x$
2. Input $y$
3. Output $out$

The scheduler should be able to used to build a function with signature $func(x, y, out)$.
Please see the following cells for usage.

In [ ]:
import tvm
import numpy as np
import sys
# Adding assignment 3 to the system path
# Make sure this matches your git directory
sys.path.insert(0, PROJECT_ROOT)
from src.ops import make_conv1d_gpu_scheduler

M = 16384
N = 32
dtype = 'float32'
a_np = np.random.rand(M).astype(dtype)
w_np = np.random.rand(N).astype(dtype)
b_np = np.convolve(a_np, w_np)

s, A, W, B = make_conv1d_gpu_scheduler(M, N)
func = tvm.build(s, [A, W, B], "cuda")

dev = tvm.cuda(0)
a = tvm.nd.array(a_np, dev)
w = tvm.nd.array(w_np, dev)
b = tvm.nd.array(np.zeros((M+N-1), dtype), dev)
func(a, w, b)
evaluator = func.time_evaluator(func.entry_name, dev, number=1, repeat =1)


print("Answer:", b_np)
print("Output:", b)
print(f"1DConv TVM: %f ms" % (evaluator(a, w, b).mean * 1e3))

Answer: [0.04649749 0.7307492  1.5013732  ... 1.1339213  0.5953749  0.06987306]
Output: [0.04649749 0.7307492  1.501373   ... 1.1339213  0.5953749  0.06987306]
1DConv TVM: 4.151264 ms


In [ ]:
print(tvm.lower(s, [A, W, B], simple_mode=True))

# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer((16384,), "float32"), W: T.Buffer((32,), "float32"), B: T.Buffer((16415,), "float32")):
        T.func_attr({"from_legacy_te_schedule": T.bool(True), "tir.noalias": T.bool(True)})
        blockIdx_x = T.launch_thread("blockIdx.x", 257)
        threadIdx_x = T.launch_thread("threadIdx.x", 64)
        if T.likely(blockIdx_x * 64 + threadIdx_x < 16415):
            B[blockIdx_x * 64 + threadIdx_x] = T.float32(0)
        for k in range(16415):
            if T.likely(blockIdx_x * 64 + threadIdx_x < 16415):
                B[blockIdx_x * 64 + threadIdx_x] = B[blockIdx_x * 64 + threadIdx_x] + T.if_then_else(16384 <= k or blockIdx_x * 64 + threadIdx_x - k < 0 or 1 <= blockIdx_x * 2 + (threadIdx_x - k) // 32, T.float32(0), A[k] * W[blockIdx_x * 64 + threadIdx_x - k])


In [ ]:
%cd {PROJECT_ROOT}
!python -m pytest tests/test_1dconv_gpu.py

/content/gdrive/MyDrive/ece5545/a3-anya-23-ct
======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.4.0
rootdir: /content/gdrive/MyDrive/ece5545/a3-anya-23-ct
plugins: anyio-3.7.1
collected 15 items                                                                                 

tests/test_1dconv_gpu.py ...............                                                     [100%]

======================================= 15 passed in 15.01s ========================================


After optimisation

In [ ]:
import tvm
import numpy as np
import sys
# Adding assignment 3 to the system path
# Make sure this matches your git directory
sys.path.insert(0, PROJECT_ROOT)
from src.ops import make_conv1d_gpu_scheduler

M = 16384
N = 32
dtype = 'float32'
a_np = np.random.rand(M).astype(dtype)
w_np = np.random.rand(N).astype(dtype)
b_np = np.convolve(a_np, w_np)

s, A, W, B = make_conv1d_gpu_scheduler(M, N)
func = tvm.build(s, [A, W, B], "cuda")

dev = tvm.cuda(0)
a = tvm.nd.array(a_np, dev)
w = tvm.nd.array(w_np, dev)
b = tvm.nd.array(np.zeros((M+N-1), dtype), dev)
func(a, w, b)
evaluator = func.time_evaluator(func.entry_name, dev, number=1, repeat =1)


print("Answer:", b_np)
print("Output:", b)
print(f"1DConv TVM: %f ms" % (evaluator(a, w, b).mean * 1e3))

Answer: [0.03542731 0.35422784 0.908636   ... 0.8905729  0.56378263 0.5235259 ]
Output: [0.03542731 0.3542278  0.908636   ... 0.8905729  0.56378263 0.5235259 ]
1DConv TVM: 0.020096 ms


In [ ]:
print(tvm.lower(s, [A, W, B], simple_mode=True))

# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer((16384,), "float32"), W: T.Buffer((32,), "float32"), B: T.Buffer((16415,), "float32")):
        T.func_attr({"from_legacy_te_schedule": T.bool(True), "tir.noalias": T.bool(True)})
        A_pad = T.allocate([16446], "float32", "global")
        A_pad_1 = T.Buffer((16446,), data=A_pad)
        with T.launch_thread("blockIdx.x", 257) as blockIdx_x:
            threadIdx_x = T.launch_thread("threadIdx.x", 64)
            if T.likely(blockIdx_x * 32 + threadIdx_x // 2 < 8223):
                A_pad_1[blockIdx_x * 64 + threadIdx_x] = T.if_then_else(blockIdx_x * 64 + threadIdx_x < 31 or 16415 <= blockIdx_x * 64 + threadIdx_x, T.float32(0), A[blockIdx_x * 64 + threadIdx_x - 31])
        blockIdx_x = T.launch_thread("blockIdx.x", 257)
        threadIdx_x = T.launch_thread("threadIdx.x", 64)
        if T.likely(blockIdx_x * 64 + threadIdx_x < 16415):
         

In [ ]:
%cd {PROJECT_ROOT}
!python -m pytest tests/test_1dconv_gpu.py

/content/gdrive/MyDrive/ece5545/a3-anya-23-ct
======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.4.0
rootdir: /content/gdrive/MyDrive/ece5545/a3-anya-23-ct
plugins: anyio-3.7.1
collected 15 items                                                                                 

tests/test_1dconv_gpu.py ...............                                                     [100%]

======================================= 15 passed in 15.21s ========================================
